# Pandas 練習  

Pandas を使ったデータ処理の練習をします。  

[なんちゃって個人情報](http://kazina.com/dummy/) で入手したダミー個人データを加工していきます。  

In [ ]:
# モジュールのインポート
import pandas as pd

In [ ]:
# 前半のダミーデータの読み込み
df_0 = pd.read_csv("./pandas練習用データ/ダミー個人データ_0.csv", encoding="shiftjis")
df_0.head()

In [ ]:
# 後半のダミーデータの読み込み
df_1 = pd.read_csv("./pandas練習用データ/ダミー個人データ_1.csv", encoding="shiftjis")
df_1.tail()

In [ ]:
# ダミーデータを縦方向に連結
df = pd.concat([df_0, df_1], ignore_index=True)
df

In [ ]:
# 電話番号と携帯で桁数がおかしいところがある
# 半角スペースになっているようなので、0 で置き換える
df["電話番号"] = df["電話番号"].str.replace(" ", "0")
df["携帯"] = df["携帯"].str.replace(" ", "0")
df.head()

In [ ]:
# Pandas は内部で Matplotlib を import しており
# DataFrame から直接グラフをプロットすることができる
df_age = df["年齢"]
df_age.hist()

In [ ]:
df["キャリア"].hist()

In [ ]:
# Matplotlib は日本語フォントを追加しなければ表示できない
# 今回は日本語名称を英字名称に置き換える
df["キャリア"].unique()

In [ ]:
# それぞれ置き換えを実施
df.loc[df["キャリア"] == "ソフトバンク", ["キャリア"]] = "SoftBank"
df.loc[df["キャリア"] == "ドコモ", ["キャリア"]] = "NTT DoCoMo"
df.loc[df["キャリア"] == "ツーカー", ["キャリア"]] = "au"  # ツーカーは au ということにする(^^)/
df["キャリア"].hist()

In [ ]:
# カレーの食べ方を正規表現を使って詳細化 (分割)
df["カレールーの位置"] = df["カレーの食べ方"].str.extract("^(.*)・(.*)$")[0]
df["カレーの攻め方"] = df["カレーの食べ方"].str.extract("^(.*)・(.*)$")[1]
df.head()

In [ ]:
# ルーの位置の種類
df["カレールーの位置"].unique()

In [ ]:
# 攻め方の種類
df["カレーの攻め方"].unique()

In [ ]:
# 年齢を元に世代に分類
bin = [10, 20, 30, 40, 50, 60, 70, 80, 100]
lab = ["10代", "20代", "30代", "40代", "50代", "60代", "70代", "80代以上"]

df["世代"] = pd.cut(df["年齢"], bins=bin, right=False, labels=lab)
df.head()

In [ ]:
# 世代の確認
df[df["世代"] == "10代"].head()

In [ ]:
# 世代の確認
df[df["世代"] == "70代"].head()

In [ ]:
# 世代毎に携帯電話を契約しているかどうかの確率を設定し
# それをもとにデータを修正する関数を定義する
import numpy as np

def to_be_uncontracted(gen_series: pd.Series, cont_prob: dict) -> pd.Series:
    
    gen_arr = gen_series.to_numpy()
    buf = []
    for gen in gen_arr:
        if np.random.rand() <= 1-prob[gen]:
            buf.append(True)
        else:
            buf.append(False)
    return pd.Series(buf)

In [ ]:
# 元のデータをコピーし、契約しているかどうかの確率を設定
df_ = df.copy()

prob = {"10代": 0.70, "20代": 0.98, "30代": 0.98, "40代": 0.97, "50代": 0.95, "60代": 0.89, "70代": 0.78, "80代以上": 0.65}

In [ ]:
# 関数を適用
df_["契約見直し"] = to_be_uncontracted(df_["世代"], prob)

In [ ]:
# 適用結果を元にデータを修正
df_.loc[df_["契約見直し"], ["キャリア"]] = "no_contract"
df_.loc[df_["契約見直し"], ["携帯"]] = np.nan

In [ ]:
# データの確認
df_[df_["キャリア"]=="no_contract"].head()

In [ ]:
# ヒストグラムでも確認
df_["キャリア"].hist()

In [ ]:
# ピボットテーブルでも確認
pd.pivot_table(df_, index="キャリア", columns="世代", values=["名前"], aggfunc="count")

In [ ]:
# 保存用のデータのためにいらない列項目を削除
df_processed = df_.drop(columns=["カレーの食べ方", "契約見直し"])
df_processed.head()

In [ ]:
# 加工済みのデータを保存
df_processed.to_csv("./pandas練習用データ/加工済みデータ.csv", encoding="utf-8")